<a href="https://colab.research.google.com/github/FutureAI22/UK-Renewable-Energy-Consumption-1990-2020/blob/main/UK_Renewable_Energy_Consumption_(1990_2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of UK Renewable Energy Consumption Trends (1990-2020)

**Project Date:** April 7, 2025
**Location Context:** Bolton, England, United Kingdom

## Introduction

This notebook explores the evolution of renewable energy consumption within the United Kingdom over a significant 30-year period, from 1990 to 2020. As the UK works towards its legally binding target of achieving Net Zero greenhouse gas emissions by 2050, understanding the historical trajectory of energy consumption and the growth of renewable sources is crucial.

**Objective:**

The primary goal of this analysis is to:
* Visualize and interpret the trends in total UK energy consumption and the fraction derived from renewable sources.
* Identify the key renewable energy technologies (Wind, Solar, Hydro, Biomass/Waste, etc.) that have driven growth during this period.
* Derive insights from the data regarding the pace and nature of the UK's energy transition up to 2020.
* Develop data-backed observations relevant to energy policy and sustainability goals.

**Dataset:**

The analysis utilizes a dataset detailing UK energy consumption, reportedly sourced via Kaggle (originally based on UK government energy statistics). The data spans the years 1990-2020 and includes:
* Total primary energy consumption.
* Total energy derived from renewable and waste sources.
* The calculated fraction of total energy from renewable and waste sources.
* A breakdown of energy generation (in Million Tonnes of Oil Equivalent - MTOE, assumed) from various renewable sources like Hydro, Wind/Wave/Tidal, Solar PV, Landfill Gas, Sewage Gas, Municipal Solid Waste (MSW), and different forms of Biomass.

**Methodology & Tools:**

This project employs Python for data analysis and visualization, primarily utilizing the following libraries:
* **Pandas:** For data loading, cleaning, manipulation, and aggregation.
* **Plotly (Plotly Express & Graph Objects):** For creating interactive visualizations to explore trends and patterns effectively.

**Structure:**

The notebook follows these main steps:
1.  Data Loading and Initial Exploration.
2.  Data Cleaning and Preparation.
3.  Analysis of Overall Energy Trends (Total Consumption, Renewable Absolute & Fraction).
4.  Analysis of Contributions from Different Renewable Sources.
5.  Creation of a Combined Dashboard Visual.
6.  Summary of Insights and Discussion of Data Limitations/Future Work.

This analysis aims to provide a clear picture of the UK's renewable energy journey during a critical period, offering insights relevant to the ongoing transition towards a more sustainable energy system.

# **Step 1: Setup and Load Data**
This first step involves importing the necessary Python libraries (pandas for data manipulation and plotly.express for interactive plotting) and loading your uk_renewable_energy.csv dataset into a pandas DataFrame. We'll also perform a quick initial exploration to see the first few rows and get information about the columns and data types.

In [1]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.io as pio # Optional: For setting default themes

# Optional: Set a default theme for Plotly plots (e.g., 'plotly_white')
pio.templates.default = "plotly_white"

# --- Load the dataset ---
# Make sure 'uk_renewable_energy.csv' is uploaded to your Colab environment
# or provide the correct path if it's in Google Drive.
file_path = 'uk_renewable_energy.csv' # Adjust if your filename is different

try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    print("Please make sure the file is uploaded to Google Colab and the path is correct.")
    # Stop execution if file not found
    exit()

# --- Initial exploration ---
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset Information (columns, data types, non-null counts):")
df.info()

Dataset loaded successfully.

First 5 rows of the dataset:
   Year  Energy from renewable & waste sources  \
0  1990                                  1.647   
1  1991                                  1.634   
2  1992                                  1.843   
3  1993                                  1.862   
4  1994                                  2.528   

   Total energy consumption of primary fuels and equivalents  \
0                                            225.532           
1                                            231.288           
2                                            228.696           
3                                            231.368           
4                                            230.739           

   Fraction from renewable sources and waste  Hydroelectric power  \
0                                      0.007                0.448   
1                                      0.007                0.398   
2                                      0.008    

# **Step 2: Data Cleaning and Preparation**
Here, we'll clean up the column names from the original CSV file, replacing spaces and special characters with underscores or shorter names to make them easier to reference in the code. We will keep 'Year' as a regular column, which works well with Plotly Express. We'll then verify the changes by displaying the new column names and the first few rows of the cleaned data.

In [2]:
# --- Clean Column Names ---
# Create a copy to avoid modifying the original DataFrame directly
df_cleaned = df.copy()

# Define a mapping for cleaner column names (adjust if needed based on df.info())
column_mapping = {
    'Year': 'Year',
    'Energy from renewable & waste sources': 'Renewable_Energy_MTOE',
    'Total energy consumption of primary fuels and equivalents': 'Total_Energy_Consumption_MTOE',
    'Fraction from renewable sources and waste': 'Renewable_Fraction',
    'Hydroelectric power': 'Hydro_MTOE',
    'Wind, wave, tidal': 'Wind_Wave_Tidal_MTOE',
    'Solar photovoltaic': 'Solar_PV_MTOE',
    'Geothermal aquifers': 'Geothermal_MTOE',
    'Landfill gas': 'Landfill_Gas_MTOE',
    'Sewage gas': 'Sewage_Gas_MTOE',
    'Biogas from autogen': 'Biogas_Autogen_MTOE', # Assuming 'autogen' means 'anaerobic digestion' or similar
    'Municipal solid waste (MSW)': 'MSW_MTOE',
    'Poultry litter': 'Poultry_Litter_MTOE',
    'Straw': 'Straw_MTOE',
    'Wood': 'Wood_MTOE',
    'Charcoal': 'Charcoal_MTOE', # Often considered biomass
    'Liquid bio-fuels': 'Liquid_Biofuels_MTOE',
    'Bioethanol': 'Bioethanol_MTOE',
    'Biodiesel': 'Biodiesel_MTOE',
    'Biomass': 'Other_Biomass_MTOE', # Check if this aggregates other biomass or is separate
    'Cross-boundary Adjustment': 'Cross_Boundary_Adjustment_MTOE'
}

# Filter the mapping to only include columns present in the DataFrame
valid_column_mapping = {k: v for k, v in column_mapping.items() if k in df_cleaned.columns}
df_cleaned.rename(columns=valid_column_mapping, inplace=True)

# --- Verify Cleaning ---
print("\nCleaned Column Names:")
print(df_cleaned.columns)

print("\nFirst 5 rows of cleaned data:")
print(df_cleaned.head())

# --- Check Data Types Again (Especially Year) ---
print("\nData types after cleaning:")
df_cleaned.info()

# Ensure 'Year' is treated as a numerical value for plotting axes
# If it's not already numeric (e.g., int64), uncomment the line below:
# df_cleaned['Year'] = pd.to_numeric(df_cleaned['Year'])


Cleaned Column Names:
Index(['Year', 'Renewable_Energy_MTOE', 'Total_Energy_Consumption_MTOE',
       'Renewable_Fraction', 'Hydro_MTOE', 'Wind_Wave_Tidal_MTOE',
       'Solar_PV_MTOE', 'Geothermal_MTOE', 'Landfill_Gas_MTOE',
       'Sewage_Gas_MTOE', 'Biogas_Autogen_MTOE', 'MSW_MTOE',
       'Poultry_Litter_MTOE', 'Straw_MTOE', 'Wood_MTOE', 'Charcoal_MTOE',
       'Liquid_Biofuels_MTOE', 'Bioethanol_MTOE', 'Biodiesel_MTOE',
       'Other_Biomass_MTOE', 'Cross_Boundary_Adjustment_MTOE'],
      dtype='object')

First 5 rows of cleaned data:
   Year  Renewable_Energy_MTOE  Total_Energy_Consumption_MTOE  \
0  1990                  1.647                        225.532   
1  1991                  1.634                        231.288   
2  1992                  1.843                        228.696   
3  1993                  1.862                        231.368   
4  1994                  2.528                        230.739   

   Renewable_Fraction  Hydro_MTOE  Wind_Wave_Tidal_MTOE  Solar

# **Step 3: Basic Trend Analysis with Plotly**
This section focuses on visualizing the main overall trends in the data using interactive line charts created with Plotly Express. We will plot:

The fraction of total energy coming from renewable sources over the years.
The absolute amount of renewable energy consumed.
The total primary energy consumption.

In [3]:
# --- Plot 1: Renewable Fraction Over Time (Interactive) ---
fig_fraction = px.line(df_cleaned,
                       x='Year',
                       y='Renewable_Fraction',
                       title='UK Renewable Energy Fraction (1990-2020)',
                       markers=True, # Add markers to data points
                       labels={'Renewable_Fraction': 'Fraction of Total Consumption', 'Year': 'Year'}) # Nicer axis labels
fig_fraction.update_layout(yaxis_tickformat='.1%') # Format y-axis as percentage
fig_fraction.show()

# --- Plot 2: Absolute Renewable Energy Over Time (Interactive) ---
fig_absolute_renew = px.line(df_cleaned,
                             x='Year',
                             y='Renewable_Energy_MTOE',
                             title='Absolute Renewable Energy Consumption (1990-2020)',
                             markers=True,
                             labels={'Renewable_Energy_MTOE': 'Energy (MTOE)', 'Year': 'Year'})
fig_absolute_renew.show()

# --- Plot 3: Total Energy Consumption Over Time (Interactive) ---
fig_total_energy = px.line(df_cleaned,
                           x='Year',
                           y='Total_Energy_Consumption_MTOE',
                           title='Total Primary Energy Consumption (1990-2020)',
                           markers=True,
                           labels={'Total_Energy_Consumption_MTOE': 'Energy (MTOE)', 'Year': 'Year'})
fig_total_energy.show()

# **Step 4: Analyzing Contributions of Different Sources (Plotly)**
In this step, we investigate the composition of the total renewable energy. We first reshape the data using pandas.melt to make it suitable for Plotly Express. Then, we create an interactive stacked area chart (plotly.express.area) to visualize how the contribution of each individual renewable source (like Hydro, Wind, Solar, different types of Biomass, etc.) to the total renewable energy has evolved from 1990 to 2020.

In [4]:
# --- Identify Renewable Source Columns ---
# Use the cleaned column names we defined earlier
source_columns = [
    'Hydro_MTOE', 'Wind_Wave_Tidal_MTOE', 'Solar_PV_MTOE',
    'Geothermal_MTOE', 'Landfill_Gas_MTOE', 'Sewage_Gas_MTOE',
    'Biogas_Autogen_MTOE', 'MSW_MTOE', 'Poultry_Litter_MTOE',
    'Straw_MTOE', 'Wood_MTOE', 'Charcoal_MTOE',
    'Liquid_Biofuels_MTOE', # Might aggregate Bioethanol/Biodiesel, check definition if needed
    # 'Bioethanol_MTOE', # Keep commented if aggregated above
    # 'Biodiesel_MTOE',  # Keep commented if aggregated above
    'Other_Biomass_MTOE' # Clarify what this includes if possible from data source
]

# Filter source columns to only those present in the dataframe
available_source_columns = [col for col in source_columns if col in df_cleaned.columns]

# --- Reshape data from wide to long format using melt ---
# We need 'Year' as an identifier variable, and the source columns as value variables
df_melted = df_cleaned.reset_index().melt(
    id_vars=['Year'],                     # Column(s) to keep as identifiers
    value_vars=available_source_columns, # Columns to unpivot
    var_name='Renewable_Source',         # Name for the new column holding source names
    value_name='Energy_MTOE'             # Name for the new column holding the values
)

print("\nFirst 5 rows of the 'melted' DataFrame for plotting:")
print(df_melted.head())

# --- Create Interactive Stacked Area Plot ---
fig_stacked_area = px.area(df_melted,
                           x='Year',
                           y='Energy_MTOE',
                           color='Renewable_Source', # Color lines/areas by source
                           line_group='Renewable_Source', # Ensure lines connect correctly for each source
                           title='Contribution of Renewable Sources to Total Renewable Energy (1990-2020)',
                           labels={'Energy_MTOE': 'Energy (MTOE)', 'Renewable_Source': 'Renewable Source'},
                           # Optional: Define a specific color sequence if desired
                           # color_discrete_sequence=px.colors.qualitative.Pastel
                           )

# Improve hover information (optional)
# fig_stacked_area.update_traces(hovertemplate='Year=%{x}<br>Source=%{fullData.name}<br>Energy (MTOE)=%{y:.2f}') # More control over hover text

fig_stacked_area.show()


First 5 rows of the 'melted' DataFrame for plotting:
   Year Renewable_Source  Energy_MTOE
0  1990       Hydro_MTOE        0.448
1  1991       Hydro_MTOE        0.398
2  1992       Hydro_MTOE        0.467
3  1993       Hydro_MTOE        0.370
4  1994       Hydro_MTOE        0.438


In [5]:
# Import necessary modules for subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- Ensure your individual figures are generated ---
# Make sure the code cells for Step 3 and Step 4 have been run
# so that fig_fraction, fig_absolute_renew, fig_total_energy,
# and fig_stacked_area are defined in your environment.
# If not, you might need to re-run those cells or encapsulate
# the plotting logic in functions.

# --- Create a 2x2 subplot grid ---
# Define titles for each subplot
subplot_titles = [
    "UK Renewable Energy Fraction",
    "Absolute Renewable Energy Consumption",
    "Total Primary Energy Consumption",
    "Contribution of Renewable Sources"
]
fig_combined = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)

# --- Add traces from each figure to the corresponding subplot ---

# Quadrant 1: Renewable Fraction (Top Left: row 1, col 1)
# Add each trace (line/marker) from the original figure
for trace in fig_fraction.data:
    fig_combined.add_trace(trace, row=1, col=1)

# Quadrant 2: Absolute Renewable Energy (Top Right: row 1, col 2)
for trace in fig_absolute_renew.data:
    fig_combined.add_trace(trace, row=1, col=2)

# Quadrant 3: Total Energy Consumption (Bottom Left: row 2, col 1)
for trace in fig_total_energy.data:
    fig_combined.add_trace(trace, row=2, col=1)

# Quadrant 4: Stacked Area - Source Contributions (Bottom Right: row 2, col 2)
# This figure might have many traces (one for each source area)
for trace in fig_stacked_area.data:
    # You might want to assign legend groups if managing legends closely
    # trace.update(legendgroup='sources', name=trace.name.replace('_MTOE','')) # Example customization
    fig_combined.add_trace(trace, row=2, col=2)


# --- Update Layout for the combined figure ---
fig_combined.update_layout(
    title_text="UK Energy Trends Overview (1990-2020)",
    height=800, # Adjust height as needed
    width=1100, # Adjust width as needed
    # Adjust legend - might need tweaking depending on appearance
    # legend_tracegroup='sources', # Example: if using legend groups for area chart
    # legend_title_text='Renewable Source', # Example
    showlegend=True # Show the combined legend
)

# Optional: Update specific axes titles if they didn't carry over well
fig_combined.update_yaxes(title_text="Fraction", tickformat='.0%', row=1, col=1)
fig_combined.update_yaxes(title_text="Energy (MTOE)", row=1, col=2)
fig_combined.update_yaxes(title_text="Energy (MTOE)", row=2, col=1)
fig_combined.update_yaxes(title_text="Energy (MTOE)", row=2, col=2)
fig_combined.update_xaxes(title_text="Year", row=2, col=1)
fig_combined.update_xaxes(title_text="Year", row=2, col=2)


# --- Show the combined figure ---
fig_combined.show()

In [6]:
# saving a HTML copy of the final interactive visual
fig_combined.write_html("uk_energy_dashboard_interactive.html")